<center><H1> Machine Learning Lab #8

<H3>  K-Nearest Neighbour (K-NN) and ID-3 Decision Tree

In [3]:
#Generic Imports

import math
import io
import re
import inspect
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns

In [4]:
#Problem specific Import
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

<H2>Questions

1. Write a python function program to demonstrate the working of the decision tree based C4.5 algorithms
without using scikit-learn library. Use following data set for building the decision tree and apply this
knowledge to classify a new sample.
The dataset has three attributes: Outlook (Sunny, Overcast, Rainy), Temperature, Humidity and Wind (Weak,
Strong). The target attribute is Play Tennis (Yes/No).
    

| Day | Outlook   | Temp. | Humidity | Wind   | Decision |
|-----|-----------|-------|----------|--------|----------|
| 1   | Sunny     | 85    | 85       | Weak   | No       |
| 2   | Sunny     | 80    | 90       | Strong | No       |
| 3   | Overcast  | 83    | 78       | Weak   | Yes      |
| 4   | Rain      | 70    | 96       | Weak   | Yes      |
| 5   | Rain      | 68    | 80       | Weak   | Yes      |
| 6   | Rain      | 65    | 70       | Strong | No       |
| 7   | Overcast  | 64    | 65       | Strong | Yes      |
| 8   | Sunny     | 72    | 95       | Weak   | No       |
| 9   | Sunny     | 69    | 70       | Weak   | Yes      |
| 10  | Rain      | 75    | 80       | Weak   | Yes      |
| 11  | Sunny     | 75    | 70       | Strong | Yes      |
| 12  | Overcast  | 72    | 90       | Strong | Yes      |
| 13  | Overcast  | 81    | 75       | Weak   | Yes      |
| 14  | Rain      | 71    | 80       | Strong | No       |

In [14]:
markdown_data = """
| Day | Outlook | Temp. | Humidity | Wind | Decision |
|-----|-----------|-------|----------|--------|----------|
| 1 | Sunny | 85 | 85 | Weak | No |
| 2 | Sunny | 80 | 90 | Strong | No |
| 3 | Overcast | 83 | 78 | Weak | Yes |
| 4 | Rain | 70 | 96 | Weak | Yes |
| 5 | Rain | 68 | 80 | Weak | Yes |
| 6 | Rain | 65 | 70 | Strong | No |
| 7 | Overcast | 64 | 65 | Strong | Yes |
| 8 | Sunny | 72 | 95 | Weak | No |
| 9 | Sunny | 69 | 70 | Weak | Yes |
| 10 | Rain | 75 | 80 | Weak | Yes |
| 11 | Sunny | 75 | 70 | Strong | Yes |
| 12 | Overcast | 72 | 90 | Strong | Yes |
| 13 | Overcast | 81 | 75 | Weak | Yes |
| 14 | Rain | 71 | 80 | Strong | No |
"""

lines = markdown_data.strip().split('\n')
data_lines = [line.strip().strip('|') for line in lines if not line.startswith('|---')]
cleaned_string = '\n'.join(data_lines)
data_file_like = io.StringIO(cleaned_string)

df = pd.read_csv(
    data_file_like,
    sep='|',
    skipinitialspace=True
)

df = df.dropna(axis=1, how='all')
df.columns = df.columns.str.strip()

try:
    df.drop('Day', axis=1, inplace=True)
except KeyError:
    pass

output_filepath = 'weather_decision.csv'
df.to_csv(output_filepath, index=False)


In [15]:
df2 = pd.read_csv('weather_decision.csv')
df2.head()

,Outlook,Temp.,Humidity,Wind,Decision
0,Sunny,85,85,Weak,No
1,Sunny,80,90,Strong,No
2,Overcast,83,78,Weak,Yes
3,Rain,70,96,Weak,Yes
4,Rain,68,80,Weak,Yes


In [18]:
def entropy(data):
    labels = data['Decision'].value_counts()
    total = len(data)
    ent = 0
    for count in labels:
        prob = count / total
        ent -= prob * math.log2(prob)
    return ent

# Split dataset based on attribute
def split_data(data, attribute, value):
    return data[data[attribute] == value].reset_index(drop=True)

In [19]:
def gain_ratio(data, attribute):
    total_entropy = entropy(data)
    values = data[attribute].unique()
    split_entropy = 0
    intrinsic_value = 0
    total = len(data)

    for val in values:
        subset = split_data(data, attribute, val)
        prob = len(subset) / total
        split_entropy += prob * entropy(subset)
        intrinsic_value -= prob * math.log2(prob) if prob > 0 else 0

    info_gain = total_entropy - split_entropy
    return info_gain / intrinsic_value if intrinsic_value != 0 else 0

In [20]:
def build_tree_c45(data, attributes):
    labels = data['Decision'].unique()
    if len(labels) == 1:
        return labels[0]

    if not attributes:
        return data['Decision'].mode()[0]

    gains = {attr: gain_ratio(data, attr) for attr in attributes}
    best_attr = max(gains, key=gains.get)

    tree = {best_attr: {}}
    for val in data[best_attr].unique():
        subset = split_data(data, best_attr, val)
        if subset.empty:
            tree[best_attr][val] = data['Decision'].mode()[0]
        else:
            remaining_attrs = [a for a in attributes if a != best_attr]
            tree[best_attr][val] = build_tree_c45(subset, remaining_attrs)
    return tree

In [21]:
def classify(tree, sample):
    if isinstance(tree, str):
        return tree
    attr = next(iter(tree))
    val = sample.get(attr)
    subtree = tree[attr].get(val)
    if subtree is None:
        return "Unknown"
    return classify(subtree, sample)

In [22]:
def main():
    attributes = ['Outlook', 'Temp.', 'Humidity', 'Wind']
    tree = build_tree_c45(df, attributes)
    print("Decision Tree:")
    print(tree)

    # Classify a new sample
    new_sample = {'Outlook': 'Sunny', 'Temp.': 72, 'Humidity': 90, 'Wind': 'Weak'}
    result = classify(tree, new_sample)
    print("\nNew Sample Classification:")
    print(result)

if __name__ == "__main__":
    main()

Decision Tree:
{'Temp.': {np.int64(85): 'No ', np.int64(80): 'No ', np.int64(83): 'Yes ', np.int64(70): 'Yes ', np.int64(68): 'Yes ', np.int64(65): 'No ', np.int64(64): 'Yes ', np.int64(72): {'Outlook': {'Sunny ': 'No ', 'Overcast ': 'Yes '}}, np.int64(69): 'Yes ', np.int64(75): 'Yes ', np.int64(81): 'Yes ', np.int64(71): 'No '}}

New Sample Classification:
Unknown


2. Write a python function program to demonstrate the working of the decision tree based CART algorithms
without using scikit-learn library. Use Q. No. 1 data set for building the decision tree and apply this knowledge
to classify a new sample.
The dataset has three attributes: Outlook (Sunny, Overcast, Rainy), Temperature, Humidity and Wind (Weak,
Strong). The target attribute is Play Tennis (Yes/No).

In [25]:
def gini_index(groups, classes):
    total_instances = sum(len(group) for group in groups)
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            proportion = group['Decision'].value_counts().get(class_val, 0) / size
            score += proportion ** 2
        gini += (1.0 - score) * (size / total_instances)
    return gini

In [26]:
def test_split(attribute, value, dataset):
    if dataset[attribute].dtype == 'object':
        left = dataset[dataset[attribute] == value]
        right = dataset[dataset[attribute] != value]
    else:
        left = dataset[dataset[attribute] <= value]
        right = dataset[dataset[attribute] > value]
    return left, right

In [27]:
def get_best_split(dataset):
    class_values = dataset['Decision'].unique()
    best_gini = float('inf')
    best_attr, best_val, best_groups = None, None, None

    for attribute in dataset.columns[:-1]:  # exclude target
        for val in dataset[attribute].unique():
            groups = test_split(attribute, val, dataset)
            gini = gini_index(groups, class_values)
            if gini < best_gini:
                best_gini, best_attr, best_val, best_groups = gini, attribute, val, groups
    return {'attribute': best_attr, 'value': best_val, 'groups': best_groups}

In [28]:
def to_terminal(group):
    return group['Decision'].mode()[0]

In [29]:
def build_tree_cart(node, max_depth, depth):
    left, right = node['groups']
    del(node['groups'])

    # Check for no split
    if left.empty or right.empty:
        node['left'] = node['right'] = to_terminal(pd.concat([left, right]))
        return

    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return

    # Left child
    if len(left['Decision'].unique()) == 1:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_best_split(left)
        build_tree_cart(node['left'], max_depth, depth + 1)

    # Right child
    if len(right['Decision'].unique()) == 1:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_best_split(right)
        build_tree_cart(node['right'], max_depth, depth + 1)


In [30]:
def predict(node, sample):
    if isinstance(node, str):
        return node
    attr, val = node['attribute'], node['value']
    if isinstance(val, str):
        branch = node['left'] if sample[attr] == val else node['right']
    else:
        branch = node['left'] if sample[attr] <= val else node['right']
    return predict(branch, sample)

In [31]:
def main2():
    root = get_best_split(df)
    build_tree_cart(root, max_depth=3, depth=1)
    print("CART Decision Tree:")
    print(root)

    # Classify a new sample
    new_sample = {'Outlook': 'Sunny', 'Temp.': 72, 'Humidity': 90, 'Wind': 'Weak'}
    result = predict(root, new_sample)
    print("\nNew Sample Classification:")
    print(result)

if __name__ == "__main__":
    main2()


CART Decision Tree:
{'attribute': 'Outlook', 'value': 'Overcast ', 'left': 'Yes ', 'right': {'attribute': 'Temp.', 'value': np.int64(75), 'left': {'attribute': 'Temp.', 'value': np.int64(65), 'left': 'No ', 'right': 'Yes '}, 'right': 'No '}}

New Sample Classification:
Yes 


3. Write a python function program to demonstrate the working of the decision tree based C4.5 and CART
algorithms without and with using scikit-learn library. Using the following dataset, apply aforementioned
algorithms. The attributes are Income (Low, Medium, High) and Credit (Good, Bad), and the target is Loan
Approved (Yes/No).

| Income  | Credit | Loan Approved |
|---------|--------|----------------|
| Low     | Good   | Yes            |
| Low     | Bad    | No             |
| Medium  | Good   | Yes            |
| Medium  | Bad    | Yes            |
| High    | Good   | Yes            |
| High    | Bad    | No             |


In [32]:
data = {
    'Income': ['Low', 'Low', 'Medium', 'Medium', 'High', 'High'],
    'Credit': ['Good', 'Bad', 'Good', 'Bad', 'Good', 'Bad'],
    'Loan Approved': ['Yes', 'No', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('loan_data.csv', index=False)

In [ ]:

def encode_column(column):
    return pd.Series(column).astype('category').cat.codes

df['Income'] = encode_column(df['Income'])
df['Credit'] = encode_column(df['Credit'])
df['Loan Approved'] = encode_column(df['Loan Approved'])

X = df[['Income', 'Credit']]  
y = df['Loan Approved']       


def entropy(y):
    prob = y.value_counts(normalize=True)
    return -np.sum(prob * np.log2(prob))

def gini_index(y):
    prob = y.value_counts(normalize=True)
    return 1 - np.sum(prob ** 2)


def information_gain(X_column, y):
    total_entropy = entropy(y)
    values = X_column.unique()
    weighted_entropy = 0
    for value in values:
        subset = y[X_column == value]
        weighted_entropy += (len(subset) / len(X_column)) * entropy(subset)
    return total_entropy - weighted_entropy

def gini_gain(X_column, y):
    total_gini = gini_index(y)
    values = X_column.unique()
    weighted_gini = 0
    for value in values:
        subset = y[X_column == value]
        weighted_gini += (len(subset) / len(X_column)) * gini_index(subset)
    return total_gini - weighted_gini

def best_split(X, y, algorithm="C4.5"):
    best_score = -float('inf')
    best_feature = None
    for feature in X.columns:
        if algorithm == "C4.5":
            score = information_gain(X[feature], y)
        elif algorithm == "CART":
            score = gini_gain(X[feature], y)
        if score > best_score:
            best_score = score
            best_feature = feature
    return best_feature


def build_tree(X, y, algorithm="C4.5"):
    if len(y.unique()) == 1:  
        return y.iloc[0]
    if len(X.columns) == 0:  
        return y.mode()[0]

    best_feature = best_split(X, y, algorithm)
    tree = {best_feature: {}}
    remaining_features = X.drop(columns=[best_feature])

    for value in X[best_feature].unique():
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]
        tree[best_feature][value] = build_tree(subset_X, subset_y, algorithm)
    
    return tree




def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree
    feature = list(tree.keys())[0]
    value = sample[feature]
    if value in tree[feature]:
        return predict(tree[feature][value], sample)
    return None  
    

c4_5_tree = build_tree(X, y, algorithm="C4.5")
cart_tree = build_tree(X, y, algorithm="CART")


test_sample = X.iloc[2]  
c4_5_prediction = predict(c4_5_tree, test_sample)
cart_prediction = predict(cart_tree, test_sample)

print("C4.5 Tree:")

print(f"C4.5 Prediction for sample {test_sample}: {c4_5_prediction}")

print("\nCART Tree:")

print(f"CART Prediction for sample {test_sample}: {cart_prediction}")

C4.5 Tree:
C4.5 Prediction for sample Income    2
Credit    1
Name: 2, dtype: int8: 1

CART Tree:
CART Prediction for sample Income    2
Credit    1
Name: 2, dtype: int8: 1


In [34]:
label_encoder = LabelEncoder()
df['Income'] = label_encoder.fit_transform(df['Income'])
df['Credit'] = label_encoder.fit_transform(df['Credit'])
df['Loan Approved'] = label_encoder.fit_transform(df['Loan Approved'])


X = df[['Income', 'Credit']]  
y = df['Loan Approved']       


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

c4_5_classifier = DecisionTreeClassifier(criterion='entropy', random_state=42)
c4_5_classifier.fit(X_train, y_train)


cart_classifier = DecisionTreeClassifier(criterion='gini', random_state=42)
cart_classifier.fit(X_train, y_train)

y_pred_c4_5 = c4_5_classifier.predict(X_test)
accuracy_c4_5 = accuracy_score(y_test, y_pred_c4_5)


y_pred_cart = cart_classifier.predict(X_test)
accuracy_cart = accuracy_score(y_test, y_pred_cart)

print(f"Accuracy of C4.5 (Entropy) Classifier: {accuracy_c4_5 * 100:.2f}%")
print(f"Accuracy of CART (Gini) Classifier: {accuracy_cart * 100:.2f}%")

Accuracy of C4.5 (Entropy) Classifier: 100.00%
Accuracy of CART (Gini) Classifier: 100.00%


---